<a href="https://colab.research.google.com/github/xquan16/AI-Study-Buddy/blob/main/AI_Study_Buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# === 1. 设置 (SETUP) ===
# (这是你每次重开笔记本时，唯一需要运行的“启动”单元格)

print("--- [1/4] 正在克隆 (Cloning) 你的 GitHub 仓库...")
# 1.1 克隆 (Clone) 你的 GitHub 仓库
!git clone https://github.com/xquan16/AI-Study-Buddy.git

print("--- [2/4] 正在安装 (Installing) 必要的“工具箱”...")
# 1.2 安装 (Install) 我们需要的所有“工具箱”
# - 'google-generativeai' = 最后的“聊天”大脑 (Gemini)
# - 'pandas' = 我们的“超级 Excel”
# - 'sentence-transformers' = 【新工具】我们自己的“向量化 AI”
!pip install google-generativeai pandas sentence-transformers

print("--- [3/4] 正在导入 (Importing) 我们的 Python 工具...")
# 1.3 导入 (Import) 我们的 Python 工具
import google.generativeai as genai
import pandas as pd
import re
import os
import time # (修复 NameError, 虽然我们现在不用 time.sleep 了)
from google.colab import userdata
from sentence_transformers import SentenceTransformer # 导入我们的“新工具”

print("--- [4/4] 正在配置 (Configuring) Gemini大脑...")
# 1.4 从“保险箱” (Secrets 🔑) 里安全地取出你的 API Key 并配置
# (我们还是先配置好，虽然现在不用它)
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)
    print("--- ✅ 设置完成！你的“实验室”已经准备就绪。 ---")
except Exception as e:
    # 就算 API Key 额度用完，也没关系，我们先用“本地” AI
    print(f"--- 警告：无法配置 Google API Key (额度已用完)。但没关系，我们先用“本地” AI。 ---")

--- [1/4] 正在克隆 (Cloning) 你的 GitHub 仓库...
fatal: destination path 'AI-Study-Buddy' already exists and is not an empty directory.
--- [2/4] 正在安装 (Installing) 必要的“工具箱”...
--- [3/4] 正在导入 (Importing) 我们的 Python 工具...
--- [4/4] 正在配置 (Configuring) Gemini大脑...
--- ✅ 设置完成！你的“实验室”已经准备就绪。 ---


In [15]:
# === 2. 读取并清洗 (READ & CLEAN) ===
# (这个单元格假设你已经运行了上面的“设置单元格”)

print("--- 正在“读取”和“清洗”你的笔记... ---")

# 2.1 定义文件路径 (它在 "Setup Cell" 克隆下来的文件夹里)
file_path = "AI-Study-Buddy/notes.txt"

# 2.2 用标准 Python 读取整个文件
with open(file_path, 'r', encoding='latin1') as f:
    full_notes_text = f.read()

# 2.3 分块 (Chunking): 按照 "?? " 切割成段落
chunks_list = full_notes_text.split("?? ")

# 2.4 清洗 (Cleaning): 去掉换行符(\n)和制表符(\t)
cleaned_chunks = []
for chunk in chunks_list:
    clean_text = re.sub(r'\s+', ' ', chunk).strip()
    if len(clean_text) > 30: # 只保留有意义的段落
        cleaned_chunks.append(clean_text)

# 2.5 把“干净的段落列表”放进一个新的 Pandas 表格 (DataFrame)
df_cleaned = pd.DataFrame(cleaned_chunks, columns=["Clean_Chunk"])

print("--- ✅ 成功：笔记已“清洗”并“分块”！ ---")
pd.set_option('display.max_colwidth', None)
print(df_cleaned.head())

--- 正在“读取”和“清洗”你的笔记... ---
--- ✅ 成功：笔记已“清洗”并“分块”！ ---
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Clean_Chunk
0                                                                                                                               

In [16]:
# === 3. 向量化 (EMBEDDINGS) - 【新方法：本地 AI】 ===
# (这个单元格假设你已经成功运行了【单元格 1】和【单元格 2】)

print("--- 步骤 3: 正在“加载”我们自己的“向量化 AI”... ---")
# 3.1. 加载“AI 品尝助手机器人” (SentenceTransformer)
#    "all-MiniLM-L6-v2" 是一个非常有名的、又快又好的模型
#    第一次运行会需要下载模型 (大概 80MB)，请稍候...
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("--- ✅ AI 已加载到 Colab！ ---")


print("--- 正在“向量化”你的所有笔记段落 (这会非常快)... ---")
# 3.2. 我们现在可以直接“批量”处理所有段落
#    我们不需要 "for" 循环和 "time.sleep" 了！

# 3.2.1. 把你表格里 "Clean_Chunk" 这一列转成一个列表 (list)
chunks_to_embed = df_cleaned['Clean_Chunk'].tolist()

# 3.2.2. 【关键】让“本地 AI”一次性“品尝”所有段落
#      这 100% 免费，而且没有限速！
embedding_list = embedding_model.encode(chunks_to_embed)

print("--- ✅ 成功：你的所有笔记段落都已“向量化”！ ---")

# 3.3. 把“风味卡”列表，作为“新的一列”添加到我们的表格里
df_embedded = df_cleaned.copy()
df_embedded['Embedding'] = embedding_list.tolist() # 转回 .tolist() 方便 Pandas 处理

# 3.4. 打印出成果
print("\n这是你“向量化”后的表格 (只显示第一行):")
print(df_embedded.head(1))

# 3.5. 让我们看看一张“风味卡”(向量) 到底长什么样
if not df_embedded.empty:
    print("\n--- 这是一张“风味卡”(向量) 的样子 (只显示前 10 个数字): ---")
    print(df_embedded.iloc[0]['Embedding'][:10])
    print(f"\n--- 这张卡片总共有 {len(df_embedded.iloc[0]['Embedding'])} 个数字。 ---")

--- 步骤 3: 正在“加载”我们自己的“向量化 AI”... ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- ✅ AI 已加载到 Colab！ ---
--- 正在“向量化”你的所有笔记段落 (这会非常快)... ---
--- ✅ 成功：你的所有笔记段落都已“向量化”！ ---

这是你“向量化”后的表格 (只显示第一行):
                                                                                                                                                                                                                                                                 Clean_Chunk  \
0  What Is K-Nearest Neighbors (KNN)?  KNN is a simple, instance-based learning algorithm used for both classification and regression.  It predicts the output for a new data point based on the outputs of the K most similar (nearest) points from the training dataset.   

                                                                                                                                                                                                                                                                                                                                                     

In [17]:
# === 4. 向量搜索 (VECTOR SEARCH) ===

import numpy as np  # 导入“数学引擎” Numpy，昵称 "np"
from scipy.spatial.distance import cosine # 导入“余弦距离”计算器

# 4.1. 定义一个函数，用来“向量化”你的“问题”
# (这个函数 *使用* 你在【单元格 3】加载的“本地 AI”)
def get_query_embedding(query_text):
    # .encode() 是我们【单元格 3】里用过的同一个命令
    return embedding_model.encode(query_text)

# 4.2. 定义一个函数，用来“搜索”
# 这是我们项目的“核心引擎”
def find_similar_chunks(query_embedding, df_embeddings, top_k=3):

    # 1. 准备好我们所有的“笔记向量”(风味卡)
    #    我们把 Pandas 表格里的 'Embedding' 列拿出来
    note_embeddings = np.array(df_embeddings['Embedding'].tolist())

    # 2. 【核心数学】计算“距离”
    #    我们用 'cosine' 距离来计算“你的问题”和“每一段笔记”的“意思相似度”
    #    "distance = 0" 意思是“意思完全一样”
    #    "distance = 1" 意思是“意思完全无关”
    distances = [cosine(query_embedding, note_vec) for note_vec in note_embeddings]

    # 3. 找出“距离最近”的 K 个
    #    np.argsort(distances) 会返回“距离”*从小到大*的“索引号 (index)”
    nearest_indices = np.argsort(distances)[:top_k]

    # 4. 返回那 K 个最相似的“原文段落”
    #    df_embeddings.iloc[nearest_indices] 是 Pandas 命令，意思是：
    #    "从我的表格里，把这 K 行 (indices) 拿出来"
    return df_embeddings.iloc[nearest_indices]

print("--- ✅ 搜索引擎已准备就绪！ ---")

# ---------------------------------
# 🚀 让我们来测试一下！
# ---------------------------------

# 4.3. 你的“问题”
my_query = "What is KNN?"

print(f"\n--- 你的问题是 (Query): \"{my_query}\" ---")

# 4.4. 把你的“问题”也变成“向量”
my_query_vector = get_query_embedding(my_query)

# 4.5. 运行我们的“搜索引擎”！
#    (我们来找 K=3 个最相关的段落)
relevant_chunks = find_similar_chunks(my_query_vector, df_embedded, top_k=3)

print("\n--- ✅ 搜索完成！AI 在你的笔记中找到了 3 个最相关的段落： ---")
pd.set_option('display.max_colwidth', None)
print(relevant_chunks)

--- ✅ 搜索引擎已准备就绪！ ---

--- 你的问题是 (Query): "What is KNN?" ---

--- ✅ 搜索完成！AI 在你的笔记中找到了 3 个最相关的段落： ---
                                                                                                                                                                                                                                                                  Clean_Chunk  \
0   What Is K-Nearest Neighbors (KNN)?  KNN is a simple, instance-based learning algorithm used for both classification and regression.  It predicts the output for a new data point based on the outputs of the K most similar (nearest) points from the training dataset.   
3                                                                                                                                                                                                                           Real-Life Use Cases: Example Type How KNN is Used   
56                                                                               

In [19]:
# === 5. 生成 (GENERATION) ===

# 5.1. 重新定义我们的“问题”
my_query = "What is KNN?"

# 5.2. 【复习】我们再次运行“向量搜索”来获取“食材”
#    (这一步只是为了让这个单元格能独立运行，它会非常快)
my_query_vector = get_query_embedding(my_query)
relevant_chunks = find_similar_chunks(my_query_vector, df_embedded, top_k=3)

print(f"--- 成功找到了 {len(relevant_chunks)} 段相关笔记... ---")

# 5.3. 【关键！】构建我们的“最终提示” (Final Prompt)
#    这是我们发给“大厨” (Gemini) 的指令

# 5.3.1. 先把我们的“食材”(3 个笔记段落) 组合成一个大字符串
context_string = ""
for index, row in relevant_chunks.iterrows():
    context_string += row['Clean_Chunk'] + "\n---\n"

# 5.3.2. 这是我们的“指令模板”
prompt_template = f"""
你是一个专业的 AI 学习助手 (AI Study Buddy)。
你的任务是根据我提供的背景资料，来回答我的问题。
请“只使用”我提供的背景资料，不要添加任何你自己的额外知识。
如果背景资料里没有答案，就直接说“根据我手上的资料，我无法回答这个问题”。

【背景资料 (My Notes)】:
{context_string}

【我的问题 (My Question)】:
{my_query}

【你的回答 (Your Answer)】:
"""

# 5.4. 调用“AI 大厨” (Gemini)
try:
    # 1. 选择“聊天”用的大脑型号
    chat_model = genai.GenerativeModel('models/gemini-pro-latest')

    # 2. 把我们精心准备的“指令”发给它
    response = chat_model.generate_content(prompt_template)

    # 3. 打印出“大厨”做好的“菜”（最终答案）
    print("--- ✅ AI Study Buddy 的最终回答: ---")
    print(response.text)

except Exception as e:
    print(f"--- 错误：调用 Gemini 聊天 API 失败 --- \n{e}")
    print("--- (这可能是因为你的 API Key 免费额度真的用完了) ---")

--- 成功找到了 3 段相关笔记... ---
--- ✅ AI Study Buddy 的最终回答: ---
根据我手上的资料：

KNN 是一种简单的、基于实例的学习算法，可用于分类和回归。它根据训练数据集中 K 个最相似（最近）的数据点的输出来预测新数据点的输出。
